In [13]:
import os
import numpy as np
import pandas as pd

## Derivative

```python
from utilities import differentiate
```

In [1]:
class DerivativeError(Exception):
  """
  Raised when there is an error in calculation of derivative
  """

  def __init__(self, message):
    super().__init__(message)

def calculate_difference(points):
  """
  Returns a list of difference between two consecutive elements of the
  `points`.

  :param points: list of elements whose differences need to be returned
  :return: list of consectutive differences of elements in `points`

  Example:
  >>> points = [1,2,3,4]
  >>> calculate_difference(points)
  [1,1,1]

  # The array is computed as - [2-1, 3-2, 4-3]
  """

  d_points = []
  for i in range(0,len(points)-1):
    p1 = points[i]
    p2 = points[i+1]
    d_points.append(p2-p1)
  return d_points

def differentiate(x, y):
  """
  Differentiates y with respect to x.

  :param x: list of absicssae
  :param y: list of ordinates
  :return: derivative of y with respect to x

  .. math::
      \frac{dx}{dy} = lim_{\Delta x \to 0} \frac{\Delta y}{\Delta x}

  This function is effective when :math:`\Delta x \approx 0 `.

  Example:
  >>> x = [1,2,3,4,5]
  >>> y = [4,5,6,7,9]
  >>> differentiate(x, y)
  [1,1,1,2]
  """

  if len(x) != len(y):
    raise DerivativeError('Unequal datapoints in the dataset')
  x = calculate_difference(x)
  y = calculate_difference(y)
  return [y[i]/x[i] for i in range(len(x))]

## Group Duplicates

```python
from utilities import group_by
```

In [ ]:
def group_by(x, y):
  """
  Attemts to convert a relation from `x` to `y` to a function from `x` to `y`
  by grouping values of `y` as mean corresponding to same `x`.

  :param x: list of absicssae
  :param y: list of ordinates
  :return: a function from `x` to `y`

  Example:
  >>> x = [1,1,1,2,3]
  >>> y = [2,3,4,4,5]
  >>> group_by(x, y)
  ([1, 2, 3], [3, 4, 5])
  """

  new_x = []
  new_y = []
  datapoints = len(x)
  count = 1
  sum = 0
  for i in range(datapoints-1):
    if x[i] == x[i+1]:
      count += 1
      sum += y[i]
    elif x[i] != x[i+1]:
      new_x.append(x[i])
      if count == 1:
        new_y.append(y[i])
      else:
        new_y.append((sum+y[i])/count)
      count = 1
      sum = 0
  if x[datapoints - 1] == x[datapoints - 2]:
    new_x.append(x[datapoints - 2])
    new_y.append((sum+y[datapoints-1])/count)
  else:
    new_x.append(x[datapoints - 1])
    new_y.append(y[datapoints - 1]/count)
  return new_x, new_y

## Find nearest element

```python
from utilities import find_nearest
```

In [10]:
def distance_formula(x, y, x1, y1):
    return (x-x1)**2 + (y-y1)**2

def find_nearest(x, y, estimated_x, estimated_y):
    """
    Returns the index in the array whose value is nearest to `value`
    :param array: list of values
    :param value: value to be found
    :return: the index in the array whose value is nearest to `value`
    """

    idx = 0
    best_so_far = distance_formula(x[idx], y[idx], estimated_x, estimated_y)
    for i in range(len(x)):
        if (best_so_far > distance_formula(x[i], y[i], estimated_x, estimated_y)):
            best_so_far = distance_formula(x[i], y[i], estimated_x, estimated_y)
            idx = i
    return idx

## Save plot and figures

In [4]:
# Function defined to save figure to the output directory
def save_fig (module, function, name):
    method_to_call = getattr(module, function)
    method_to_call(name)

## Index and output file

In [16]:
class FileNotOpen(Exception):
    def __init__(self, filename):
        message = f'{filename} not opened'
        return super().__init__(message)


class File:
    def __init__(self, file_path, message):
        self.file_path = file_path
        self.file_object = None

        if os.path.exists(self.file_path):
            if File.choose_Y_n(message):
                os.unlink(self.file_path)
                with open(self.file_path, 'w'):
                    pass
        else:
            with open(file_path, 'w'):
                pass

    @staticmethod
    def choose_Y_n(message):
        choose = input(f'{message} (Y/n): ').lower()
        if choose in ['y', 'yes']:
            return True
        return False

    def close_file(self):
        if self.file_object is not None:
            self.file_object.close()
        else:
            raise FileNotOpen(self.NAME)
            

class OutputFile(File):
    NAME = 'output'

    def open_file(self, mode='a'):
        self.file_object = open(self.file_path, mode)

    def save_parameter (self, name, variable, units):
        self.open_file()
        self.file_object.write(f'{name} = {variable} {units}\n')
        self.close_file()


class IndexFile(File):
    NAME = 'index'
    COLUMN_NAME = 'Parameters'
    SEPARATOR = ','

    def open_file(self):
        self.file_object = pd.read_csv(self.file_path, sep=self.SEPARATOR, names=[self.COLUMN_NAME])

    def exists_parameter(self, parameter):
        query_frame = self.file_object.loc[self.file_object[self.COLUMN_NAME] == parameter]
        if query_frame.empty:
            return False
        return True

    def add_parameter(self, parameter):
        self.open_file()
        if not self.exists_parameter(parameter):
            self.file_object = self.file_object.append({self.COLUMN_NAME: parameter}, ignore_index=True)
            self.file_object.to_csv(self.file_path, sep=self.SEPARATOR, header=False, index=False)
            return True
        return False